In [15]:
from file_utils import *
from model_factory import get_model
from dataset_factory import get_datasets

import sys, os
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from datetime import datetime
from torch.cuda.amp import GradScaler, autocast

In [16]:
config_data = read_file_in_dir(sys.path[0], 'default_test' + '.json')
vocab, train_loader, val_loader, test_loader = get_datasets(config_data)
model = get_model(config_data, vocab)
criterion = nn.CrossEntropyLoss()

loaded vocab
Loaded Dataset
Loaded Model


In [17]:
state_dict = torch.load(os.path.join('experiment_data/baseline_1', 'latest_model.pt'))
model.load_state_dict(state_dict['model'])

<All keys matched successfully>

In [18]:
def convert_question(prediction):
        """
        Converts predicted question indices to word tokens
        prediction: N x Q
        """
        word_idxs = prediction.cpu().numpy()
        captions = []
        for i in range(prediction.shape[0]):
            words = [vocab.idx2word[idx].lower() for idx in word_idxs[i]]
            try:
                end_idx = words.index('<end>') + 1 # cut off after predicting end
            except ValueError as e:
                end_idx = None
            
            words = words[:end_idx]
            captions.append(words)
        
        to_return = []
        for i in range(len(captions)):
            clean_list = ['<pad>', '<start>', '<end>', '<unk>', ' ', ';', ',', '.', '\'', '-', '(', ')', '[', ']', '@', '$', \
                '%', '!', '?', '/', '+', '^', '&', '*']
            cleaned_caption = [word for word in captions[i] if word not in clean_list]
            to_return.append(cleaned_caption)

        return to_return

In [19]:
model.temperature = config_data['generation']['temperature']
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

with torch.no_grad():
    for i, (passages, answers, questions) in enumerate(test_loader):
        if torch.cuda.is_available:
            passages = passages.cuda().long()
            answers = answers.cuda().long()
            questions = questions.cuda().long()

        # Metric Evaluation
        predictions = model.predict(passages, answers) # N x Q
        predictions = convert_question(predictions) # list of lists of tokens
        true_questions = convert_question(questions) # list of lists of tokens
        break

print(' '.join(predictions[0]))
print(' '.join(true_questions[0]))
print(' '.join(predictions[1]))
print(' '.join(true_questions[1]))
print(' '.join(predictions[2]))
print(' '.join(true_questions[2]))

what what the
at what time is the harvard yale rivalry set aside
what what the the
what type of education was assessed during this time
what what the the
the adaptive immune system must distinguish between what types of molecules
